In [1]:
import pandas as pd
import os
from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt
from ccx0526 import ccx_pre
from consumer0526 import consu_pre
from behavior0526 import beha_pre
from FeatureEngineer2 import FeatureEngineer
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
from sklearn import cross_validation, metrics
from sklearn.grid_search import GridSearchCV
from sklearn.model_selection import train_test_split
import matplotlib.pylab as plt
%matplotlib inline

def modelfit(alg, dtrain, dtest, useTrainCV=True, cv_folds=5, early_stopping_rounds=50):

    if useTrainCV:
        xgb_param = alg.get_xgb_params()
        xgtrain = xgb.DMatrix(dtrain.ix[:,2:], label=dtrain['target'])
        xgtest = xgb.DMatrix(dtest.ix[:,2:])
        cvresult = xgb.cv(xgb_param, xgtrain, num_boost_round=alg.get_params()['n_estimators'], nfold=cv_folds,
             early_stopping_rounds=early_stopping_rounds)
        alg.set_params(n_estimators=cvresult.shape[0])
        print(cvresult.shape[0])
    
    #建模
    alg.fit(dtrain.ix[:,2:], dtrain['target'],eval_metric='auc')
        
    #对训练集预测
    dtest_predictions = alg.predict(dtest.ix[:,2:])
    dtest_predprob = alg.predict_proba(dtest.ix[:,2:])[:,1]
        
    #输出模型的一些结果
    #print ("\n关于现在这个模型")
    #print ("准确率 : %.4g" % metrics.accuracy_score(dtest['target'].values, dtest_predictions))
    print ("AUC 得分 (测试集): %f" % metrics.roc_auc_score(dtest['target'], dtest_predprob))

D:\Anaconda\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
D:\Anaconda\lib\site-packages\sklearn\grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [2]:
target=pd.read_csv("../data/train_scene_A/train_target_A.csv")
ccx_data=pd.read_csv("../data/train_scene_A/train_ccx_A.csv")
consumer=pd.read_csv("../data/train_scene_A/train_consumer_A.csv")
behaviorA = pd.read_csv("../data/train_scene_A/train_behavior_A.csv")
behaviorB = pd.read_csv("../data/train_scene_B/train_behavior_B.csv")

D:\Anaconda\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
D:\Anaconda\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (13,14) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
print(ccx_data.shape) ##53983*7
data1=ccx_pre(ccx_data)
print(data1.shape) ##9389*30

(53983, 7)
(9389, 30)


In [4]:
data=pd.merge(target,data1,how='left')
print(data.shape)

xgb1 = XGBClassifier(
        learning_rate =0.1,
        n_estimators=1000,
        max_depth=6,
        min_child_weight=1,
        gamma=0,
        subsample=0.7,
        colsample_bytree=0.8,
        objective= 'binary:logistic')

train, test = train_test_split(data, random_state=2)
predictors = [x for x in train.columns if x not in ['target', 'ccx_id']]
modelfit(xgb1, train, test)

(21245, 31)


D:\Anaconda\lib\site-packages\ipykernel_launcher.py:22: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix
D:\Anaconda\lib\site-packages\ipykernel_launcher.py:23: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix


7
AUC 得分 (测试集): 0.588847


D:\Anaconda\lib\site-packages\sklearn\preprocessing\label.py:171: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [5]:
print(consumer.shape) ##268453*15
data2=consu_pre(consumer)
print(data2.shape) ##21245*48

(268453, 15)


D:\Anaconda\lib\site-packages\pandas\core\generic.py:3549: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)
C:\Users\steven.wang\Desktop\风云杯\code\consumer0526.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  dis_data['V_1']=[i if i in index1 else 'else' for i in dis_data['V_1']]
C:\Users\steven.wang\Desktop\风云杯\code\consumer0526.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/in

C:\Users\steven.wang\Desktop\风云杯\code\consumer0526.py:91: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  date_data['seconds_interval']=[i.total_seconds() for i in date_data['time_interval']]
C:\Users\steven.wang\Desktop\风云杯\code\consumer0526.py:93: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  date_data['seconds_interval'][(date_data['days_interval']>100) | (date_data['days_interval']<0)]=np.nan


(21245, 48)


In [6]:
data=pd.merge(target,data2,how='left')
print(data.shape)
train, test = train_test_split(data, random_state=2)
modelfit(xgb1, train, test)

(21245, 49)


D:\Anaconda\lib\site-packages\ipykernel_launcher.py:22: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix
D:\Anaconda\lib\site-packages\ipykernel_launcher.py:23: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix


7
AUC 得分 (测试集): 0.536128


D:\Anaconda\lib\site-packages\sklearn\preprocessing\label.py:171: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [13]:
data2[['consumer_count','V_4', 'V_5', 'V_6', 'V_9', 'V_10', 'V_12', 'V_13','seconds_interval_x',
       'seconds_interval_y']].head()

,consumer_count,V_4,V_5,V_6,V_9,V_10,V_12,V_13,seconds_interval_x,seconds_interval_y
0,20,1.000000,62.748500,63.748500,0.0,77.764500,1.157895,45.572435,4254.000000,13.0
1,3,1.666667,12.866667,14.533333,0.0,4.166667,1.000000,12.866667,64.666667,23.0
2,6,1.000000,372.950000,373.950000,0.0,7.416667,1.000000,372.950000,25258.000000,25258.0
3,5,0.000000,31.018000,31.018000,0.0,182.982000,1.000000,31.018000,135.400000,25.0
4,5,0.000000,33.964000,33.964000,0.0,90.936000,1.000000,26.774000,1024.200000,19.0


In [14]:
data2.shape

(21245, 48)

In [7]:
fe = FeatureEngineer(data2)
fe.discretize('consumer_count',bins=6)
names2=['V_5', 'V_6', 'V_10', 'V_13','seconds_interval_x','seconds_interval_y']
for var2 in names2:
    fe.discretize(var2,bins=10)

In [11]:
print(data2.shape)
data2.head()

(21245, 55)


,ccx_id,consumer_count,V_1_P0,V_1_P1,V_1_P11,V_1_P19,V_1_P2,V_1_P22,V_1_P4,V_1_P6,...,V11_period_none,seconds_interval_x,seconds_interval_y,consumer_count_category,V_5_category,V_6_category,V_10_category,V_13_category,seconds_interval_x_category,seconds_interval_y_category
0,1,20,10.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.550000,4254.000000,13.0,4,4,4,6,5.0,9.0,4.0
1,2,3,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,...,0.000000,64.666667,23.0,1,0,0,1,0.0,6.0,6.0
2,4,6,0.0,0.0,0.0,0.0,0.0,0.0,6.0,0.0,...,0.166667,25258.000000,25258.0,2,9,9,1,9.0,9.0,9.0
3,8,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,...,0.000000,135.400000,25.0,2,1,1,8,3.0,7.0,7.0
4,10,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,...,0.000000,1024.200000,19.0,2,1,1,6,2.0,8.0,6.0


In [8]:
data=pd.merge(target,data2,how='left')
print(data.shape)
train, test = train_test_split(data, random_state=2)
predictors = [x for x in train.columns if x not in ['target', 'ccx_id']]
modelfit(xgb1, train, test)
##预测效果下降

(21245, 56)


D:\Anaconda\lib\site-packages\ipykernel_launcher.py:22: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix
D:\Anaconda\lib\site-packages\ipykernel_launcher.py:23: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix


7
AUC 得分 (测试集): 0.524797


D:\Anaconda\lib\site-packages\sklearn\preprocessing\label.py:171: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [9]:
print(behaviorA.shape) ##21245*2271
data3,_=beha_pre(behaviorA,behaviorB)
print(data3.shape) ##21245*185

(21245, 2271)
begin to compute the missing rate......
begin to delete the heavily missing variables......
begin to deal with var1-var10......


C:\Users\steven.wang\Desktop\风云杯\code\behavior0526.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  b["q_var1"][b["var1"]>29]=1
C:\Users\steven.wang\Desktop\风云杯\code\behavior0526.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  b["q_var1"][b["var1"]>39]=2


begin to deal with year variables......


C:\Users\steven.wang\Desktop\风云杯\code\behavior0526.py:77: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  b["var16"][b["var16"]=="nan"]= np.nan
C:\Users\steven.wang\Desktop\风云杯\code\behavior0526.py:79: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  b["var16"][b["var16"]==2018]=0


begin to imputate the float and get dummy the object
(21245, 184)


In [8]:
data3.head()

,ccx_id,var1,var2,var7,var8,var9,var10,var16,var17,var155,...,var12_DT2,var12_DT3,var12_DT4,var12_DT5,var12_DT6,var12_DT7,var12_DT8,var12_DT9,var18_GC1,var18_GC2
0,1,26,0,0.0,0.0,0.0,1.0,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
1,2,28,0,0.0,0.0,0.0,1.0,NaN,NaN,66.0,...,0,0,0,0,0,0,0,0,0,0
2,4,40,1,0.0,0.0,1.0,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
3,8,21,1,0.0,0.0,0.0,1.0,NaN,NaN,112.0,...,0,0,0,0,0,0,0,0,0,0
4,10,33,1,0.0,0.0,0.0,1.0,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0


In [10]:
data=pd.merge(target,data3,how='left')
print(data.shape)
train, test = train_test_split(data, random_state=2)
predictors = [x for x in train.columns if x not in ['target', 'ccx_id']]
modelfit(xgb1, train, test, predictors)

(21245, 185)


D:\Anaconda\lib\site-packages\ipykernel_launcher.py:22: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix
D:\Anaconda\lib\site-packages\ipykernel_launcher.py:23: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix


7
AUC 得分 (测试集): 0.582477


D:\Anaconda\lib\site-packages\sklearn\preprocessing\label.py:171: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [19]:
data3.columns

Index(['ccx_id', 'var1', 'var2', 'var7', 'var8', 'var9', 'var16', 'var17',
       'var155', 'var156',
       ...
       'var12_DT3', 'var12_DT4', 'var12_DT5', 'var12_DT6', 'var12_DT7',
       'var12_DT8', 'var12_DT9', 'var18_GC1', 'var18_GC2', 'var1_category'],
      dtype='object', length=185)

In [24]:
fe = FeatureEngineer(data3)
#fe.discretize('consumer_count',bins=6)
names2=['var1']
for var2 in names2:
    #print(var2)
    fe.discretize(var2,bins=10)

var1


In [25]:
names2=['var16','var17']
for var2 in names2:
    #print(var2)
    fe.discretize(var2,bins=5)

In [27]:
data=pd.merge(target,data3,how='left')
print(data.shape)
train, test = train_test_split(data, random_state=2)
predictors = [x for x in train.columns if x not in ['target', 'ccx_id']]
modelfit(xgb1, train, test, predictors) ##提高1%

(21245, 188)


D:\Anaconda\lib\site-packages\ipykernel_launcher.py:22: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix
D:\Anaconda\lib\site-packages\ipykernel_launcher.py:23: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix


7
AUC 得分 (测试集): 0.593322


D:\Anaconda\lib\site-packages\sklearn\preprocessing\label.py:171: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [28]:
data2=consu_pre(consumer)
data=pd.merge(data1,data2,how='right')
data=pd.merge(data,data3)
data=pd.merge(target,data)
print(data.shape)
data.head()

D:\Anaconda\lib\site-packages\pandas\core\generic.py:3549: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)
C:\Users\steven.wang\Desktop\风云杯\code\consumer0526.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  dis_data['V_1']=[i if i in index1 else 'else' for i in dis_data['V_1']]
C:\Users\steven.wang\Desktop\风云杯\code\consumer0526.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/in

C:\Users\steven.wang\Desktop\风云杯\code\consumer0526.py:91: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  date_data['seconds_interval']=[i.total_seconds() for i in date_data['time_interval']]
C:\Users\steven.wang\Desktop\风云杯\code\consumer0526.py:93: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  date_data['seconds_interval'][(date_data['days_interval']>100) | (date_data['days_interval']<0)]=np.nan


(21245, 264)


,ccx_id,target,var_01_C2,var_01_C3,var_02_T1,var_02_T2,var_02_T3,var_02_T5,var_02_T6,var_02_T7,...,var12_DT5,var12_DT6,var12_DT7,var12_DT8,var12_DT9,var18_GC1,var18_GC2,var1_category,var16_category,var17_category
0,4711,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,8,NaN,NaN
1,9078,0,2.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,NaN,NaN
2,4631,0,4.0,3.0,0.0,1.0,2.0,1.0,2.0,0.0,...,0,0,0,0,0,1,0,4,NaN,3.0
3,40018,0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0,0,0,0,0,0,0,9,NaN,NaN
4,25082,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,7,NaN,NaN


In [ ]:
xgb1 = XGBClassifier(
        learning_rate =0.1,
        n_estimators=1000,
        max_depth=6,
        min_child_weight=1,
        gamma=0,
        subsample=0.7,
        colsample_bytree=0.8,
        objective= 'binary:logistic')

train, test = train_test_split(data, random_state=1)
predictors = [x for x in train.columns if x not in ['target', 'ccx_id']]
modelfit(xgb1, train, test)

D:\Anaconda\lib\site-packages\ipykernel_launcher.py:22: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix
D:\Anaconda\lib\site-packages\ipykernel_launcher.py:23: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix


## 建模与交叉验证

写一个大的函数完成以下的功能
1. 数据建模
2. 求测试准确率
3. 求测试集AUC
4. 根据xgboost交叉验证更新n_estimators
5. 画出特征的重要度 

## 第1步- 对于高的学习率找到最合适的estimators个数

In [30]:
#predictors = [x for x in train.columns if x not in [target, IDcol]]
xgb1 = XGBClassifier(
        learning_rate =0.1,
        n_estimators=1000,
        max_depth=6,
        min_child_weight=1,
        gamma=0,
        subsample=0.7,
        colsample_bytree=0.8,
        objective= 'binary:logistic',
        nthread=4,
        scale_pos_weight=1)

In [31]:
train, test = train_test_split(data, random_state=0)
modelfit(xgb1, train, test)

D:\Anaconda\lib\site-packages\ipykernel_launcher.py:22: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix
D:\Anaconda\lib\site-packages\ipykernel_launcher.py:23: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix


70
AUC 得分 (测试集): 0.610980


D:\Anaconda\lib\site-packages\sklearn\preprocessing\label.py:171: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [49]:
#对subsample 和 max_features 用grid search查找最好的参数
param_test1 = {
    'max_depth':list(range(4,10,1)),
    'min_child_weight':list(range(1,11,2))
}
gsearch1 = GridSearchCV(estimator = XGBClassifier( learning_rate =0.1, n_estimators=140, max_depth=5,
                                        min_child_weight=1, gamma=0, subsample=0.8, colsample_bytree=0.8,
                                        objective= 'binary:logistic', nthread=4, scale_pos_weight=1, seed=27), 
                       param_grid = param_test1, scoring='roc_auc',n_jobs=4,iid=False, cv=5)
gsearch1.fit(data[col],data[target])

Exception ignored in: <bound method DMatrix.__del__ of <xgboost.core.DMatrix object at 0x000002C476278748>>
Traceback (most recent call last):
  File "D:\Anaconda\lib\site-packages\xgboost\core.py", line 366, in __del__
    if self.handle is not None:
AttributeError: 'DMatrix' object has no attribute 'handle'
Exception ignored in: <bound method DMatrix.__del__ of <xgboost.core.DMatrix object at 0x000002C4218606A0>>
Traceback (most recent call last):
  File "D:\Anaconda\lib\site-packages\xgboost\core.py", line 366, in __del__
    if self.handle is not None:
AttributeError: 'DMatrix' object has no attribute 'handle'


GridSearchCV(cv=5, error_score='raise',
       estimator=XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=5, min_child_weight=1, missing=None, n_estimators=140,
       n_jobs=1, nthread=4, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=27, silent=True,
       subsample=0.8),
       fit_params={}, iid=False, n_jobs=4,
       param_grid={'max_depth': [4, 6, 8, 10, 12], 'min_child_weight': [1, 3, 5, 7, 9]},
       pre_dispatch='2*n_jobs', refit=True, scoring='roc_auc', verbose=0)

In [50]:
gsearch1.best_params_, gsearch1.best_score_

({'max_depth': 4, 'min_child_weight': 5}, 0.5897101513319175)

In [51]:
#Grid seach选择合适的lr
param_test2 = {
    'learning_rate':[i/20 for i in range(1,6)]
}
gsearch2 = GridSearchCV(estimator = XGBClassifier( learning_rate =0.1, n_estimators=140, max_depth=4,
                                        min_child_weight=5, gamma=0, subsample=0.8, colsample_bytree=0.8,
                                        objective= 'binary:logistic', nthread=4, scale_pos_weight=1,seed=27), 
                       param_grid = param_test2, scoring='roc_auc',n_jobs=4,iid=False, cv=5)
gsearch2.fit(data[col],data[target])

GridSearchCV(cv=5, error_score='raise',
       estimator=XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=4, min_child_weight=5, missing=None, n_estimators=140,
       n_jobs=1, nthread=4, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=27, silent=True,
       subsample=0.8),
       fit_params={}, iid=False, n_jobs=4,
       param_grid={'learning_rate': [0.05, 0.1, 0.15, 0.2, 0.25]},
       pre_dispatch='2*n_jobs', refit=True, scoring='roc_auc', verbose=0)

In [52]:
gsearch2.best_params_, gsearch2.best_score_

({'learning_rate': 0.05}, 0.5942362164290684)

In [55]:
#Grid seach选择合适的n_est
param_test3 = {
    'n_estimators':[i*30 for i in range(1,5)]
}
gsearch3 = GridSearchCV(estimator = XGBClassifier( learning_rate =0.05, n_estimators=140, max_depth=4,
                                        min_child_weight=5, gamma=0, subsample=0.8, colsample_bytree=0.8,
                                        objective= 'binary:logistic', nthread=4, scale_pos_weight=1,seed=27), 
                       param_grid = param_test3, scoring='roc_auc',n_jobs=4,iid=False, cv=5)
gsearch3.fit(data[col],data[target])

GridSearchCV(cv=5, error_score='raise',
       estimator=XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, gamma=0, learning_rate=0.05, max_delta_step=0,
       max_depth=4, min_child_weight=5, missing=None, n_estimators=140,
       n_jobs=1, nthread=4, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=27, silent=True,
       subsample=0.8),
       fit_params={}, iid=False, n_jobs=4,
       param_grid={'n_estimators': [30, 60, 90, 120]},
       pre_dispatch='2*n_jobs', refit=True, scoring='roc_auc', verbose=0)

In [56]:
gsearch3.best_params_, gsearch3.best_score_

({'n_estimators': 60}, 0.5954678962383474)

In [57]:
#Grid seach选择合适的gamma
param_test3 = {
    'gamma':[i/10 for i in range(0,5)]
}
gsearch3 = GridSearchCV(estimator = XGBClassifier( learning_rate =0.05, n_estimators=60, max_depth=4,
                                        min_child_weight=5, gamma=0, subsample=0.8, colsample_bytree=0.8,
                                        objective= 'binary:logistic', nthread=4, scale_pos_weight=1,seed=27), 
                       param_grid = param_test3, scoring='roc_auc',n_jobs=4,iid=False, cv=5)
gsearch3.fit(data[col],data[target])

GridSearchCV(cv=5, error_score='raise',
       estimator=XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, gamma=0, learning_rate=0.05, max_delta_step=0,
       max_depth=4, min_child_weight=5, missing=None, n_estimators=60,
       n_jobs=1, nthread=4, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=27, silent=True,
       subsample=0.8),
       fit_params={}, iid=False, n_jobs=4,
       param_grid={'gamma': [0.0, 0.1, 0.2, 0.3, 0.4]},
       pre_dispatch='2*n_jobs', refit=True, scoring='roc_auc', verbose=0)

In [58]:
gsearch3.best_params_, gsearch3.best_score_

({'gamma': 0.1}, 0.5956204846749038)

In [59]:
#对subsample 和 colsample_bytree用grid search寻找最合适的参数
param_test4 = {
    'subsample':[i/10.0 for i in range(6,10)],
    'colsample_bytree':[i/10.0 for i in range(6,10)]
}
gsearch4 = GridSearchCV(estimator = XGBClassifier( learning_rate =0.05, n_estimators=60, max_depth=4,
                                        min_child_weight=5, gamma=0.1, subsample=0.8, colsample_bytree=0.8,
                                        objective= 'binary:logistic', nthread=4, scale_pos_weight=1,seed=27), 
                       param_grid = param_test4, scoring='roc_auc',n_jobs=4,iid=False, cv=5)
gsearch4.fit(data[col],data[target])

GridSearchCV(cv=5, error_score='raise',
       estimator=XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, gamma=0.1, learning_rate=0.05,
       max_delta_step=0, max_depth=4, min_child_weight=5, missing=None,
       n_estimators=60, n_jobs=1, nthread=4, objective='binary:logistic',
       random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=27, silent=True, subsample=0.8),
       fit_params={}, iid=False, n_jobs=4,
       param_grid={'subsample': [0.6, 0.7, 0.8, 0.9], 'colsample_bytree': [0.6, 0.7, 0.8, 0.9]},
       pre_dispatch='2*n_jobs', refit=True, scoring='roc_auc', verbose=0)

In [60]:
gsearch4.best_params_, gsearch4.best_score_

({'colsample_bytree': 0.6, 'subsample': 0.6}, 0.5959682400407498)

In [61]:
#对reg_alpha用grid search寻找最合适的参数
param_test6 = {
    'reg_alpha':[1e-8, 1e-7,1e-6]
}
gsearch6 = GridSearchCV(estimator = XGBClassifier( learning_rate =0.05, n_estimators=60, max_depth=4,
                                        min_child_weight=5, gamma=0.1, subsample=0.7, colsample_bytree=0.7,
                                        objective= 'binary:logistic', nthread=4, scale_pos_weight=1,seed=27), 
                       param_grid = param_test6, scoring='roc_auc',n_jobs=4,iid=False, cv=5)
gsearch6.fit(data[col],data[target])

GridSearchCV(cv=5, error_score='raise',
       estimator=XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.7, gamma=0.1, learning_rate=0.05,
       max_delta_step=0, max_depth=4, min_child_weight=5, missing=None,
       n_estimators=60, n_jobs=1, nthread=4, objective='binary:logistic',
       random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=27, silent=True, subsample=0.7),
       fit_params={}, iid=False, n_jobs=4,
       param_grid={'reg_alpha': [1e-08, 1e-07]}, pre_dispatch='2*n_jobs',
       refit=True, scoring='roc_auc', verbose=0)

In [62]:
gsearch6.best_params_, gsearch6.best_score_

({'reg_alpha': 1e-08}, 0.5953290859364506)